In [10]:
#모듈Import 
import torch
import torch.nn as nn
import torch.nn.functional as F

In [11]:
#Module class 정의
class InceptionModule(nn.Module):
    def __init__(self, in_channels):
        super(InceptionModule, self).__init__()
        
        # 1x1 Convolution
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=1)
        
        # 1x1 Conv followed by 3x3 Conv
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=1),
            nn.Conv2d(64, 128, kernel_size=3, padding=1)
        )
        
        # 1x1 Conv followed by 5x5 Conv
        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=1),
            nn.Conv2d(64, 128, kernel_size=5, padding=2)
        )
        
        # 3x3 MaxPooling followed by 1x1 Conv
        self.pool = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_channels, 32, kernel_size=1)
        )
        
    def forward(self, x):
        conv1_out = self.conv1(x)
        conv3_out = self.conv3(x)
        conv5_out = self.conv5(x)
        pool_out = self.pool(x)
        
        # Concatenate along channel dimension
        output = torch.cat([conv1_out, conv3_out, conv5_out, pool_out], dim=1)
        
        return output

In [12]:
# Create a simple model using the Inception module
class SimpleInception(nn.Module):
    def __init__(self):
        super(SimpleInception, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.inception1 = InceptionModule(64)
        self.fc = nn.Linear(64*4*32*32, 10)  # Assuming input size is [32, 32, 3]
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.inception1(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [13]:
# Create the model
model = SimpleInception()